In [6]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from typing import Any, Tuple, Dict, Set, List
import pandas as pd
import json
import ipaddress

In [ ]:
import importlib

import android
import util
import ios
import classes

# Reload the util module
importlib.reload(util)

# Reload the ios module
importlib.reload(ios)

# Reload the classes module
importlib.reload(classes)


# Reload the android module
importlib.reload(android)

In [8]:
from typing import List
from ios import (
    read_mapping_file,
    parse_iOS_permission_results,
    get_ios_dataset,
    analyze_ios,
    find_apps_to_rerun,
    create_mapping_file_for_rerun,
    compare_192_168_0_161,
    get_logs_but_no_traffic,
    get_nologs_but_traffic,
    get_all_scanning,
    get_all_local_addresses,
    add_scan_type_for_logs,
    add_results_from_first_run,
    get_prompt_stats,
    get_mappings_from_app_ids,
    bonjour_without_string,
    search_for_bonjour,
    get_apps_released_after_permission,
    get_number_of_apps_only_airplay_or_airprint,
    logs_but_no_traffic,
    manual_logs_no_traffic
)
from classes import Result, Analysis, ScanType
from util import categorice_apps, to_classify, remove_false_positive, is_phase_scanning, get_all_multicast_addresses, remove_wrong_addresses, get_stats, get_matching_results, get_dataset_ids_from_file, local_network_ranges, get_scanning_results, write_app_ids_to_file, add_cpp_matches,is_app_scanning, get_result_map, get_other_local_addresse, get_apps_for_rerun, add_results_from_first_run, get_googlecast_queries, get_airplay_queries
from android import get_android_dataset, analyze_android, analyze_android_app



In [53]:
ios_random_id_file = "../../data/dataset/ios/ios_random.txt"
ios_top_id_file = "../../data/dataset/ios/ios_popular.txt" 

black_mapping = "../../data/dataset/ios/mapping/apps_004_black.json"
white_mapping = "../../data/dataset/ios/mapping/apps_005_white.json"
red_mapping = "../../data/dataset/ios/mapping/apps_001_red.json"

permission_result_file = "../../data/dataset/ios/mapping/plist_results/2024_01_20_all.njson"


app_matches_file = "../../data/dataset/matching_in_dataset.json"
pickle_folder = "../../data/dynamic_analysis/"


In [13]:
ios_random_ids = get_dataset_ids_from_file(ios_random_id_file)
ios_top_ids = get_dataset_ids_from_file(ios_top_id_file)

In [54]:
red_mapping = read_mapping_file(
    red_mapping
)
white_mapping = read_mapping_file(
    white_mapping
)
black_mapping = read_mapping_file(
    black_mapping
)

In [ ]:
all_permission = parse_iOS_permission_results(
    permission_result_file
)

In [17]:
with open(app_matches_file, "r") as f:
    ios_to_android = json.load(f)

In [18]:
android_to_ios = {}
for k,v in ios_to_android.items():
    android_to_ios[v] = k

In [19]:
def get_value_format(result, total):
    return f"{result} ({result/total*100:.2f}%)"


def get_row(dataset_results: List[Result], format=False, only_overall=False, overall_value=None, airplay_or_multicast = False):
    (
        s_total,
        ni_total,
        i_total,
        only_ni_total,
        only_i_total,
        scanning,
        no_interaction,
        interaction,
        only_no_interaction,
        only_interaction,
    ) = get_stats(dataset_results, add_local_or_arp=True, airplay_or_multicast = airplay_or_multicast)
    if format:
        result = [
            get_value_format(s_total, overall_value)]
        
        if not only_overall:
            result.extend([
            get_value_format(i_total - only_i_total, s_total),
            get_value_format(only_i_total, s_total),
            get_value_format(only_ni_total, s_total),
            ])

        result.append(get_value_format(scanning.get(str(ScanType.local_or_arp)), overall_value))

        if not only_overall:
            result.extend([
            get_value_format(
                interaction.get(str(ScanType.local_or_arp))
                - only_interaction.get(str(ScanType.local_or_arp)),
                scanning.get(str(ScanType.local_or_arp)),
            ),
            get_value_format(
                only_interaction.get(str(ScanType.local_or_arp)), scanning.get(str(ScanType.local_or_arp))
            ),
            get_value_format(
                only_no_interaction.get(str(ScanType.local_or_arp), 0),
                scanning.get(str(ScanType.local_or_arp)),
            )
            ])


        result.append(get_value_format(scanning.get(str(ScanType.broadcast)), overall_value))

        if not only_overall:
            result.extend([
            get_value_format(
                interaction.get(str(ScanType.broadcast))
                - only_interaction.get(str(ScanType.broadcast)),
                scanning.get(str(ScanType.broadcast)),
            ),
            get_value_format(
                only_interaction.get(str(ScanType.broadcast)),
                scanning.get(str(ScanType.broadcast)),
            ),
            get_value_format(
                only_no_interaction.get(str(ScanType.broadcast), 0),
                scanning.get(str(ScanType.broadcast)),
            )])

        result.append(get_value_format(scanning.get(str(ScanType.multicast_or_airplay)), overall_value))

        if not only_overall:
            result.extend([
            get_value_format(
                interaction.get(str(ScanType.multicast_or_airplay))
                - only_interaction.get(str(ScanType.multicast_or_airplay)),
                scanning.get(str(ScanType.multicast_or_airplay)),
            ),
            get_value_format(
                only_interaction.get(str(ScanType.multicast_or_airplay)),
                scanning.get(str(ScanType.multicast_or_airplay)),
            ),
            get_value_format(
                only_no_interaction.get(str(ScanType.multicast_or_airplay), 0),
                scanning.get(str(ScanType.multicast_or_airplay)),
            ),
        ])
        return result
    else:
        result = [s_total]
        if not only_overall:
            result.extend([
            i_total - only_i_total,
            only_i_total,
            only_ni_total])
        

        result.append(scanning.get(str(ScanType.local_or_arp)))

        if not only_overall:
            result.extend([
            interaction.get(str(ScanType.local_or_arp))
            - only_interaction.get(str(ScanType.local_or_arp)),
            only_interaction.get(str(ScanType.local_or_arp)),
            only_no_interaction.get(str(ScanType.local_or_arp), 0)
            ])

        result.append(scanning.get(str(ScanType.broadcast)))

        if not only_overall:
            result.extend([
            interaction.get(str(ScanType.broadcast))
            - only_interaction.get(str(ScanType.broadcast)),
            only_interaction.get(str(ScanType.broadcast)),
            only_no_interaction.get(str(ScanType.broadcast), 0)])
        

        result.append(scanning.get(str(ScanType.multicast_or_airplay)))
        if not only_overall:
            result.extend([
            interaction.get(str(ScanType.multicast_or_airplay))
            - only_interaction.get(str(ScanType.multicast_or_airplay)),
            only_interaction.get(str(ScanType.multicast_or_airplay)),
            only_no_interaction.get(str(ScanType.multicast_or_airplay), 0)
        ])
        return result


def get_all_matching_results(
    all: List[Result], list_matching_id: List[str]
) -> List[Result]:
    result: List[Result] = []
    for app in all:
        if app.package_id in list_matching_id:
            result.append(app)

    return result


def create_table_for_platform(
    top: List[Result], random: List[Result], matches: List[Result], format=False, only_overall=False, overall_values: List[int] = None, airplay_or_multicast = False
):
    header_names = [
        "Total"
    ]
    if not only_overall:
        header_names.extend(["Both",
        "Interaction",
        "No Interaction"])
    header = pd.MultiIndex.from_product(
        [["All", "Arp", "Broadcast", "Multicast"], header_names]
    )

    rows = []
    # rows.append(get_row(top + random + matches))
    rows.append(get_row(top, format=format, only_overall=only_overall, overall_value=overall_values[0], airplay_or_multicast = airplay_or_multicast))
    rows.append(get_row(random, format=format, only_overall=only_overall, overall_value=overall_values[1], airplay_or_multicast = airplay_or_multicast))
    rows.append(get_row(matches, format=format, only_overall=only_overall, overall_value=overall_values[2], airplay_or_multicast = airplay_or_multicast))
    # print(rows)

    # "All",
    return pd.DataFrame(
        rows, index=["Top", "Random", "Matches"], columns=header
    )  # , columns=header


def create_overall_table(
    android_top: List[Result],
    android_random: List[Result],
    android_matches: List[Result],
    ios_top: List[Result],
    ios_random: List[Result],
    ios_matches: List[Result],
    format=False,
    only_overall=False,
    overall_values: List[int] = None,
    airplay_or_multicast = False
):
    android_table = create_table_for_platform(
        android_top, android_random, android_matches, format=format, only_overall=only_overall, overall_values=overall_values, airplay_or_multicast = airplay_or_multicast
    )
    ios_table = create_table_for_platform(
        ios_top, ios_random, ios_matches, format=format, only_overall=only_overall, overall_values=overall_values[3:], airplay_or_multicast = airplay_or_multicast
    )

    return pd.concat([android_table, ios_table], keys=["Android", "iOS"])


def get_rows_for_dataset(dataset: List[Result], overall_value: int = None,     airplay_or_multicast = False):
    keys = [ScanType.broadcast, ScanType.local_or_arp, ScanType.multicast_or_airplay] #local and arp
    rows = []
    (
        s_total,
        ni_total,
        i_total,
        only_ni_total,
        only_i_total,
        scanning,
        no_interaction,
        interaction,
        only_no_interaction,
        only_interaction,
    ) = get_stats(dataset, add_local_or_arp=True, airplay_or_multicast = airplay_or_multicast) 
    rows.append(
        [
            get_value_format(only_i_total, s_total),
            get_value_format(ni_total, s_total),
        ]
    )
    for key in keys:
        rows.append(
            [
                get_value_format(
                    only_interaction.get(str(key)),scanning.get(str(key))),
                get_value_format(
                    no_interaction.get(str(key)), scanning.get(str(key))),
            ]
        )
    return rows



def create_matching_table_for_platform(dataset: List[Result], overall_value: int = None,     airplay_or_multicast = False):
    index = ["Interaction",
        "No Interaction"]
    header =["All", "Broadcast", "Local", "Multicast"]
    
    rows = get_rows_for_dataset(dataset, overall_value=overall_value,     airplay_or_multicast = airplay_or_multicast)
    return pd.DataFrame(
        rows,
        index=header,
        columns=index
    ).transpose()




def create_matching_table(
    android_matches: List[Result],
    ios_matches: List[Result],
    overall_value: int = None,
    airplay_or_multicast = False
):

    android_table = create_matching_table_for_platform(android_matches,  overall_value=overall_value,airplay_or_multicast = airplay_or_multicast)
    ios_table = create_matching_table_for_platform(ios_matches,  overall_value=overall_value, airplay_or_multicast = airplay_or_multicast)

    return pd.concat([android_table, ios_table], keys=["Android", "iOS"])

In [20]:
def add_type(ios, android, result_dict):
    if str(ScanType.arp) in str(ios.no_interaction + ios.interaction) and str(ScanType.arp) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.arp)] = result_dict.get(str(ScanType.arp), 0) + 1
    if str(ScanType.local) in str(ios.no_interaction + ios.interaction) and str(ScanType.local) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.local)] = result_dict.get(str(ScanType.local), 0) + 1
    if str(ScanType.multicast) in str(ios.no_interaction + ios.interaction) and str(ScanType.multicast) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.multicast)] = result_dict.get(str(ScanType.multicast), 0) + 1
    if str(ScanType.broadcast) in str(ios.no_interaction + ios.interaction) and str(ScanType.broadcast) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.broadcast)] = result_dict.get(str(ScanType.broadcast), 0) + 1
    if str(ScanType.local_or_arp) in str(ios.no_interaction + ios.interaction) and str(ScanType.local_or_arp) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.local_or_arp)] = result_dict.get(str(ScanType.local_or_arp), 0) + 1  
    if str(ScanType.multicast_or_airplay) in str(ios.no_interaction + ios.interaction) and str(ScanType.multicast_or_airplay) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.multicast_or_airplay)] = result_dict.get(str(ScanType.multicast_or_airplay), 0) + 1  
    return result_dict


def add_type_only_interaction_first(ios, android, result_dict):
    if str(ScanType.arp) in str(ios.no_interaction + ios.interaction) and str(ScanType.arp) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.arp)] = result_dict.get(str(ScanType.arp), 0) + 1
    if str(ScanType.local) in str(ios.no_interaction + ios.interaction) and str(ScanType.local) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.local)] = result_dict.get(str(ScanType.local), 0) + 1
    if str(ScanType.multicast) in str(ios.no_interaction + ios.interaction) and str(ScanType.multicast) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.multicast)] = result_dict.get(str(ScanType.multicast), 0) + 1
    if str(ScanType.broadcast) in str(ios.no_interaction + ios.interaction) and str(ScanType.broadcast) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.broadcast)] = result_dict.get(str(ScanType.broadcast), 0) + 1
    if str(ScanType.local_or_arp) in str(ios.no_interaction + ios.interaction) and str(ScanType.local_or_arp) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.local_or_arp)] = result_dict.get(str(ScanType.local_or_arp), 0) + 1  
    if str(ScanType.multicast_or_airplay) in str(ios.no_interaction + ios.interaction) and str(ScanType.multicast_or_airplay) in str(android.no_interaction + android.interaction):
        result_dict[str(ScanType.multicast_or_airplay)] = result_dict.get(str(ScanType.multicast_or_airplay), 0) + 1  
    return result_dict


def add_type_app(app, result_dict):
    if str(ScanType.arp) in str(app.no_interaction + app.interaction):
        result_dict[str(ScanType.arp)] = result_dict.get(str(ScanType.arp), 0) + 1
    if str(ScanType.local) in str(app.no_interaction + app.interaction):
        result_dict[str(ScanType.local)] = result_dict.get(str(ScanType.local), 0) + 1
    if str(ScanType.multicast) in str(app.no_interaction + app.interaction) :
        result_dict[str(ScanType.multicast)] = result_dict.get(str(ScanType.multicast), 0) + 1
    if str(ScanType.broadcast) in str(app.no_interaction + app.interaction):
        result_dict[str(ScanType.broadcast)] = result_dict.get(str(ScanType.broadcast), 0) + 1  
    if str(ScanType.local_or_arp) in str(app.no_interaction + app.interaction) :
        result_dict[str(ScanType.local_or_arp)] = result_dict.get(str(ScanType.local_or_arp), 0) + 1
    if str(ScanType.multicast_or_airplay) in str(app.no_interaction + app.interaction):
        result_dict[str(ScanType.multicast_or_airplay)] = result_dict.get(str(ScanType.multicast_or_airplay), 0) + 1  
    return result_dict

def one_to_one_comparison(results: Dict[Result, Result]) -> (int, int, int, Dict[ScanType, int], Dict[ScanType, int], Dict[ScanType, int]):
    both = 0
    both_type = {}
    only_ios = 0
    only_ios_type = {}
    only_android = 0
    only_android_type = {}

    android_no_interactio_ios_interaction = 0
    ios_no_interaction_android_interaction = 0
    both_no_interaction = 0

    test = 0
    

    for ios, android in results.items():
        if is_app_scanning(ios):
            if is_app_scanning(android):
                both += 1
                both_type = add_type(ios, android, both_type)
                if is_phase_scanning(ios.no_interaction) and not is_phase_scanning(android.no_interaction):
                    ios_no_interaction_android_interaction += 1
                
                if is_phase_scanning(android.no_interaction) and not is_phase_scanning(ios.no_interaction):
                    android_no_interactio_ios_interaction += 1
                
                if is_phase_scanning(android.no_interaction) and is_phase_scanning(ios.no_interaction):
                    both_no_interaction += 1

                if not is_phase_scanning(android.no_interaction) and not is_phase_scanning(ios.no_interaction):
                    test += 1
            else:
                only_ios += 1
                only_ios_type= add_type_app(ios, only_ios_type)

        elif is_app_scanning(android):
            only_android += 1
            only_android_type= add_type_app(android, only_android_type)
    print(test)
    return both, only_ios, only_android, both_type, only_ios_type, only_android_type, android_no_interactio_ios_interaction, ios_no_interaction_android_interaction, both_no_interaction


def get_value_format(result, total):
    return f"{result} ({result/total*100:.2f}%)"

def create_one_to_one_comparison_table(results: Dict[Result, Result], number_of_apps = len(ios_to_android)):
    (both, only_ios, only_android, both_type, only_ios_type, only_android_type,android_no_interaction, ios_no_interaction, both_no_interaction) = one_to_one_comparison(results)
    index = ["Android", "iOS", "Both"]
    headers = ["Access", "Broadcast", "Local", "Multicast", "No Interaction Both"]

    data = [
        [
            get_value_format(only_android, only_android + only_ios + both),
            get_value_format(only_android_type.get(str(ScanType.broadcast), 0), only_android_type.get(str(ScanType.broadcast), 0) + only_ios_type.get(str(ScanType.broadcast), 0) + both_type.get(str(ScanType.broadcast), 0)),
            get_value_format(only_android_type.get(str(ScanType.local_or_arp), 0), only_android_type.get(str(ScanType.local_or_arp), 0) + only_ios_type.get(str(ScanType.local_or_arp), 0) + both_type.get(str(ScanType.local_or_arp), 0)),
            get_value_format(only_android_type.get(str(ScanType.multicast), 0), only_android_type.get(str(ScanType.multicast), 0) + only_ios_type.get(str(ScanType.multicast), 0) + both_type.get(str(ScanType.multicast), 0)),
            get_value_format(android_no_interaction, ios_no_interaction + android_no_interaction +both_no_interaction),

        ],
        [
            get_value_format(only_ios, only_android + only_ios + both),
            get_value_format(only_ios_type.get(str(ScanType.broadcast), 0), only_android_type.get(str(ScanType.broadcast), 0) + only_ios_type.get(str(ScanType.broadcast), 0) + both_type.get(str(ScanType.broadcast), 0)),
            get_value_format(only_ios_type.get(str(ScanType.local_or_arp), 0), only_android_type.get(str(ScanType.local_or_arp), 0) + only_ios_type.get(str(ScanType.local_or_arp), 0) + both_type.get(str(ScanType.local_or_arp), 0)),
            get_value_format(only_ios_type.get(str(ScanType.multicast), 0), only_android_type.get(str(ScanType.multicast), 0) + only_ios_type.get(str(ScanType.multicast), 0) + both_type.get(str(ScanType.multicast), 0)),
                        get_value_format(ios_no_interaction, ios_no_interaction + android_no_interaction +both_no_interaction),

        ],
        [
            get_value_format(both, number_of_apps),
            get_value_format(both_type.get(str(ScanType.broadcast), 0), number_of_apps),
            get_value_format(both_type.get(str(ScanType.local_or_arp), 0), number_of_apps),
            get_value_format(both_type.get(str(ScanType.multicast), 0), number_of_apps),
            get_value_format(both_no_interaction, number_of_apps),

        ]
    ]

    df = pd.DataFrame(data, index=index, columns=headers)
    return df



In [21]:
def create_bar_chart(dataset_results: List[Result], iOS: bool = False, add_local_and_arp: bool = False) -> plt:
    """
    Create a bar chart based on the dataset results.

    Args:
        dataset_results (List[Result]): The list of results from the dataset.
        iOS (bool, optional): Flag indicating if the dataset is from iOS. Defaults to False.

    Returns:
        plt: The matplotlib plot object.
    """
    barWidth: float = 0.2
    s_total: int
    ni_total: int
    i_total: int
    only_ni_total: int
    only_i_total: int
    scanning: dict
    no_interaction: dict
    interaction: dict
    only_no_interaction: dict
    only_interaction: dict
    (
        s_total,
        ni_total,
        i_total,
        only_ni_total,
        only_i_total,
        scanning,
        no_interaction,
        interaction,
        only_no_interaction,
        only_interaction,
    ) = get_stats(dataset_results, add_local_and_arp)

    # set heights of bars
    bar_scanning: List[int] = [s_total]
    bar_no_interaction: List[int] = [ni_total]
    bar_interaction: List[int] = [i_total]
    bar_only_no_interaction: List[int] = [only_ni_total]
    bar_only_interaction: List[int] = [only_i_total]
    labels: List[str] = ["All", "Arp", "Broadcast", "Multicast"]
    keys: List[ScanType] = [ScanType.arp, ScanType.broadcast, ScanType.multicast]
    if add_local_and_arp:
        keys.remove(ScanType.arp)
        keys.insert(0, ScanType.local_or_arp)


    for scan in keys:
        if iOS and scan == ScanType.arp:
            scan = ScanType.local

        if add_local_and_arp and (str(scan) == str(ScanType.local) or str(scan) == str(ScanType.arp)):
            continue
        
        
        bar_scanning.append(scanning.get(str(scan), 0))
        bar_no_interaction.append(no_interaction.get(str(scan), 0))
        bar_interaction.append(interaction.get(str(scan), 0))
        bar_only_no_interaction.append(only_no_interaction.get(str(scan), 0))
        bar_only_interaction.append(only_interaction.get(str(scan), 0))

    # Set position of bar on X axis
    r1: np.ndarray = np.arange(len(bar_scanning))
    r2: np.ndarray = [x + barWidth for x in r1]
    r3: np.ndarray = [x + barWidth for x in r2]


    # Make the plot
    plt.bar(
        r1,
        bar_scanning,
        color="gray",
        width=barWidth,
        edgecolor="black",
        label="Scanning",
    )
    plt.bar(
        r2,
        bar_no_interaction,
        color="#d3d3d3",
        width=barWidth,
        edgecolor="black",
        label="No Interaction",
    )
    plt.bar(
        r2,
        bar_only_no_interaction,
        color="#d3d3d3",
        hatch="\\",
        width=barWidth,
        edgecolor="black",
        label="Only no Interaction",
    )
    plt.bar(
        r3,
        bar_interaction,
        color="white",
        width=barWidth,
        edgecolor="black",
        label="Interaction",
    )
    plt.bar(
        r3,
        bar_only_interaction,
        color="white",
        hatch="/",
        width=barWidth,
        edgecolor="black",
        label="Only Interaction",
    )
    plt.rcParams["text.usetex"] = True
    # Add xticks on the middle of the group bars
    plt.xticks([r + barWidth for r in range(len(bar_scanning))], labels)
    plt.ylim(0, s_total+10)
    plt.legend()
    return plt

In [22]:
def create_matching_bar_chart(android_results: List[Result], ios_results, add_local_and_arp: bool = True) -> plt:
    """
    Create a bar chart based on the dataset results.

    Args:
        dataset_results (List[Result]): The list of results from the dataset.
        iOS (bool, optional): Flag indicating if the dataset is from iOS. Defaults to False.

    Returns:
        plt: The matplotlib plot object.
    """
    barWidth: float = 0.2
    s_total: int
    ni_total: int
    i_total: int
    only_ni_total: int
    only_i_total: int
    scanning: dict
    no_interaction: dict
    interaction: dict
    only_no_interaction: dict
    only_interaction: dict
    (
        s_total,
        ni_total,
        i_total,
        only_ni_total,
        only_i_total,
        scanning,
        no_interaction,
        interaction,
        only_no_interaction,
        only_interaction,
    ) = get_stats(android_results, add_local_and_arp)

    (
        s_total_ios,
        ni_total_ios,
        i_total_ios,
        only_ni_total_ios,
        only_i_total_ios,
        scanning_ios,
        no_interaction_ios,
        interaction_ios,
        only_no_interaction_ios,
        only_interaction_ios,
    ) = get_stats(ios_results, add_local_and_arp)

    # set heights of bars
    bar_no_interaction_android: List[int] = [ni_total]
    bar_interaction_android: List[int] = [i_total]
    bar_only_no_interaction_android: List[int] = [only_ni_total]
    bar_only_interaction_android: List[int] = [only_i_total]

    bar_no_interaction_ios: List[int] = [ni_total_ios]
    bar_interaction_ios: List[int] = [i_total_ios]
    bar_only_no_interaction_ios: List[int] = [only_ni_total_ios]
    bar_only_interaction_ios: List[int] = [only_i_total_ios]

    labels: List[str] = ["All", "Arp", "Broadcast", "Multicast"]
    keys: List[ScanType] = [ScanType.arp, ScanType.broadcast, ScanType.multicast_or_airplay]
    if add_local_and_arp:
        keys.remove(ScanType.arp)
        keys.insert(0, ScanType.local_or_arp)
    print(no_interaction_ios)
    print(interaction_ios)

    for scan in keys:

        if add_local_and_arp and (str(scan) == str(ScanType.local) or str(scan) == str(ScanType.arp)):
            continue
        
        bar_no_interaction_android.append(no_interaction.get(str(scan), 0))
        bar_no_interaction_ios.append(no_interaction_ios.get(str(scan), 0))

        bar_interaction_android.append(interaction.get(str(scan), 0))
        bar_interaction_ios.append(interaction_ios.get(str(scan), 0))

        bar_only_no_interaction_android.append(only_no_interaction.get(str(scan), 0))
        bar_only_no_interaction_ios.append(only_no_interaction_ios.get(str(scan), 0))

        bar_only_interaction_android.append(only_interaction.get(str(scan), 0))
        bar_only_interaction_ios.append(only_interaction_ios.get(str(scan), 0))

    # Set position of bar on X axis
    r1: np.ndarray = np.arange(len(bar_no_interaction_android))
    r2: np.ndarray = [x + barWidth for x in r1]

    r3: np.ndarray = [x + barWidth + 0.05 for x in r2]
    r4: np.ndarray = [x + barWidth  for x in r3]


    # Make the plot
    plt.bar(
        r1,
        bar_no_interaction_android,
        color="#d3d3d3",
        width=barWidth,
        edgecolor="black",
        label="Android",
    )
    plt.bar(
        r1,
        bar_only_no_interaction_android,
        color="#d3d3d3",
        hatch="/",
        width=barWidth,
        edgecolor="black",
    )
    plt.bar(
        r2,
        bar_no_interaction_ios,
        color="white",
        width=barWidth,
        edgecolor="black",
        label="iOS",
    )
    plt.bar(
        r2,
        bar_only_no_interaction_ios,
        color="white",
        hatch="/",
        width=barWidth,
        edgecolor="black",
    )

    plt.bar(
        r3,
        bar_interaction_android,
        color="#d3d3d3",
        width=barWidth,
        edgecolor="black",
    )
    plt.bar(
        r3,
        bar_only_interaction_android,
        color="#d3d3d3",
        hatch="\\",
        width=barWidth,
        edgecolor="black",
    )
    plt.bar(
        r4,
        bar_interaction_ios,
        color="white",
        width=barWidth,
        edgecolor="black",
    )
    plt.bar(
        r4,
        bar_only_interaction_ios,
        color="white",
        hatch="\\",
        width=barWidth,
        edgecolor="black",
    )
    plt.rcParams["text.usetex"] = True
    # Add xticks on the middle of the group bars
    plt.xticks([r + (barWidth+ 0.05)/2 for r in r2], labels)
    plt.ylim(0, s_total_ios+5)
    plt.legend()
    return plt

In [23]:
def get_other_local_ips(dataset_results: List[Result], my_ip_cidr: str):
    stats: Dict[str, int] = {}
    for result in dataset_results:
        if result.app_id == "com.legrandgroup.c300x":
            # scans the whole subnet - just add one
            stats["192.168.3.123"] = 1
            continue
        current = get_other_local_addresse(result.contacted_ip_addresses, my_ip_cidr= my_ip_cidr)
        
        for ip in current:
            stats[ip] = stats.get(ip, 0) + 1
    return stats





def merge_stats(stats: List[Dict[str, int]]):
    result: Dict[str, int] = {}
    for stat in stats:
        for key, value in stat.items():
            result[key] = result.get(key, 0) + value
    return result


def get_number_of_apps_with_other_local_ips(dataset_results: List[Result], my_ip_cidr: str, ios_first_run: List[Result] = None):
    result = 0
    for app in dataset_results:
        if len(get_other_local_addresse(app.contacted_ip_addresses, my_ip_cidr= my_ip_cidr)) > 0:
            result = result + 1
    return result

def get_apps_with_other_local_ips(dataset_results: List[Result], my_ip_cidr: str):
    result = set()
    for app in dataset_results:
        if len(get_other_local_addresse(app.contacted_ip_addresses, my_ip_cidr= my_ip_cidr)) > 0:
            result.add(app.app_id)
    return result

def create_table_other_local_network(dataset_results: List[List[Result]], names: List[str], my_ip_cidr: str = "192.168.2.1/24", cut_off: int = 3, add_all = False, format=False):
    all_stats = []
    addresses_above_cut_off = set()
    for result in dataset_results:
        stats = get_other_local_ips(result, my_ip_cidr)
        all_stats.append(stats)

    if add_all:
        names.append("All")
        total = merge_stats(all_stats)
        all_stats.append(total)

    for stat in all_stats:
        for k,v in stat.items():
            if v > cut_off:
                addresses_above_cut_off.add(k)


    header = ["Subnetwork"] + names
    rows = []
    for local_network in local_network_ranges:
        row = [str(local_network)]
        i = 0
        for stats in all_stats:
            count = 0
            for ip, value in stats.items():
                if ipaddress.IPv4Address(ip) in local_network:
                    count += value
            if format:
                row.append(f"{count}({count/len(dataset_results[i])*100:.2f}%)")
            else:
                row.append(count)
            i = i + 1
        rows.append(row)
        for ip in addresses_above_cut_off:
            if ipaddress.IPv4Address(ip) in local_network:
                row = [ip]
                i = 0
                for stats in all_stats:
                    if format:
                        row.append(f"{stats.get(ip, 0)}({stats.get(ip, 0)/len(dataset_results[i])*100:.2f}%)")
                    else:
                        row.append(stats.get(ip, 0))
                    i = i + 1
                rows.append(row)

        row_other_addresses = ["Other" ] # + str(local_network)
        i = 0
        for stats in all_stats:
            count = 0
            for ip, value in stats.items():
                if ip not in addresses_above_cut_off and ipaddress.IPv4Address(ip) in local_network:
                    count += value
            if format:
                row_other_addresses.append(f"{count}({count/len(dataset_results[i])*100:.2f}%)")
            else:
                row_other_addresses.append(count)
            i = i + 1
        rows.append(row_other_addresses)

    return pd.DataFrame(rows, columns=header)

def get_matching_app_numbers(android_apps, ios_apps):
    counted_both_ios = set()
    counted_both_android = set()
    both =  0
    for app in ios_apps:
        android_id = ios_to_android.get(app, "")
        if android_id in android_apps:
            counted_both_ios.add(app)
            counted_both_android.add(android_id)
            both = both + 1
    
    for app in android_apps:
        ios_id = android_to_ios.get(app, "")
        if ios_id in ios_apps:
            if ios_id not in counted_both_ios:
                counted_both_ios.add(ios_id)
                counted_both_android.add(app)
                both = both + 1
    


    ios = set(ios_apps) - set(counted_both_ios)
    android = set(android_apps) - set(counted_both_android)
    return both, len(ios), len(android)

def create_chart_for_other_local_network(dataset_results: List[Result], my_ip_cidr: str, second_dataset: List[Result]= None):

    stats = get_other_local_ips(dataset_results, my_ip_cidr)
    if second_dataset:
        stats_second = get_other_local_ips(second_dataset, my_ip_cidr)
        all_keys = list(set(stats.keys()).union(set(stats_second.keys())))
        all_keys = sorted(all_keys, key = ipaddress.IPv4Address)
        list_first = []
        list_second = []
        for key in all_keys:
            list_first.append(stats.get(key, 0))
            list_second.append(stats_second.get(key, 0))
            
        if "192.168.3.123" in all_keys:
            # Value for scanning app (not directly aded /24 to sort keys)
            index = all_keys.index("192.168.3.123")
            all_keys.remove("192.168.3.123")
            all_keys.insert(index, "192.168.3.1/24")

        barWidth: float = 0.3 


        android_apps = get_apps_with_other_local_ips(dataset_results,"192.168.2.252/24")
        ios_apps = get_apps_with_other_local_ips(second_dataset,"192.168.2.252/24")
        print(len(ios_apps))
        print(len(android_apps))
        both, ios, android = get_matching_app_numbers(android_apps, ios_apps)

        r1 = []
        r2 = []
        for i in range(len(all_keys)):
            
            if list_first[i] > 0 and list_second[i] > 0:
                r1.append(i)
                r2.append(i+barWidth)
            elif list_first[i] > 0:
                r1.append(i+ barWidth/2)
                r2.append(i)# does not matter is empty
            else:
                r2.append(i+barWidth/2)
                r1.append(i) # does not matter is empty
        plt.bar(r1,  list_first, barWidth, label="Android", color="gray", edgecolor="black")
        plt.bar(r2 , list_second, barWidth, label = "iOS", color="white", edgecolor="black")
        plt.xticks([r + barWidth/2 for r in range(len(all_keys))], all_keys)
        plt.rcParams["text.usetex"] = True
        plt.legend()
        print("both")
        print(get_value_format(both, both + ios + android))
        print("ios")
        print(get_value_format(ios, both + ios + android))
        print("android")
        print(get_value_format(android, both + ios + android))

    else:
        plt.bar(stats.keys(), stats.values())
    plt.xticks(rotation=90)
    return plt

In [24]:
def classify_multicast(dataset: List[Result]):
    result = []
    for r in dataset:
        current = set()
        try:
            if str(ScanType.multicast) in str(r.no_interaction + r.interaction):
                #https://www.iana.org/assignments/multicast-addresses/multicast-addresses.xhtml
                for address in r.contacted_ip_addresses:
                    if ipaddress.IPv4Address(address) in ipaddress.IPv4Network(
                        "224.0.0.0/4", strict=False
                    ):
                        if address == "224.0.0.22":
                            #Internet Group Management Protocol (IGMP) Version 3
                            current.add("IGMPv3")
                            pass
                        elif address == "224.0.0.251":
                            #Multicast DNS (mDNS) address
                            current.add("mDNS")
                            pass
                        elif address == "239.255.255.250":
                            #Simple Service Discovery Protocol (SSDP) address
                            current.add("SSDP")
                            pass
                        elif address == "229.255.255.250":
                            current.add("SSDP_229")
                            pass
                        elif address == "224.0.1.187":
                            # All CoAP Nodes
                            current.add("CoAP discovery")
                            pass
                        elif address == "224.0.0.7":
                            #Routing Information Protocol (RIP) ST Routers
                            current.add("RIP")
                            pass
                        elif address == "224.0.0.113":
                            # AllJoyn
                            current.add("AllJoyn")
                            pass
                        elif address == "239.195.255.251" or address == "239.255.255.251":
                            # Multicast Discovery of DNS Services
                            current.add("DNS discovery")
                            pass
                        elif address == "239.255.255.253" :
                            #SLPv2 Protocol
                            current.add("SLPv2")
                            pass
                        elif address == "224.0.1.55":
                            # extended-sys
                            current.add("extended-sys")
                            pass
                        elif address in ["239.22.16.117", "239.22.5.70", "239.22.14.37", "239.22.2.87", "239.22.8.201", "239.22.8.204", "239.22.3.105", "239.22.21.0", "239.22.17.93", "239.22.17.28", "239.22.9.57", "239.22.11.168", "239.22.14.226", "239.22.15.150", "239.22.17.80", "239.22.13.16", "239.15.255.255", "239.22.19.0", "239.22.16.116", "239.22.4.45", "239.22.1.5", "239.22.9.30", "239.22.9.215", "239.0.255.255", "239.22.18.0", "239.22.10.192", "239.22.15.79", "239.22.14.248", "239.22.6.105", "239.22.20.0", "239.14.255.255", "239.22.8.215", "239.22.7.0", "239.22.0.0", "239.22.12.2", "239.22.15.102", "224.76.78.75", "229.236.236.236", "239.143.255.250", "239.255.102.18", "233.89.188.1", "224.0.0.130", "239.22.16.157", "225.0.0.222", "239.22.238.238", "239.1.1.255"]:
                            # other
                            current.add("other")
                            pass
                        else:
                            print(address)
            if str(ScanType.airplay) in str(r.no_interaction + r.interaction):
                #ip.src == 192.168.1.146 and ip.dst == 224.0.0.251
                current.add("Airplay")
                
        except:
            pass
        result.append(current)
    return result

In [25]:
def summarize_multicast_results(multicast_apps: List[str]) -> Dict[str, int]:
    result = {}
    for app in multicast_apps:
        for multicast in app:
            result[multicast] = result.get(multicast, 0) + 1
    return result

def create_multicast_table(datasets: List[List[Result]], multicasts: List[int], names: List[str], format=False):
    all_multicast = []
    all_datasets =  []

    for dataset in datasets:
        #print(classify_multicast(dataset))
        all_multicast.append(summarize_multicast_results(classify_multicast(dataset)))
        all_datasets = all_datasets + dataset
    
    all = summarize_multicast_results(classify_multicast(all_datasets))

    header = ["Multicast"] + names
    rows = []
    for key, value in all.items():
        row = [key]
        
        for i in range(len(names)):
            v = all_multicast[i].get(key, 0)
            if format:
                row.append(f"{v}({v/multicasts[i]*100:.2f}%)")
            else:
                
                row.append(v)
        rows.append(row)
    return pd.DataFrame(rows, columns=header)

In [26]:
def get_background_multicast(dataset: List[Result]) -> Dict[str, int]:
    result = {}
    for app in dataset:
        if str(ScanType.airplay) in str(app.no_interaction + app.interaction):
            result["Airplay"] = result.get("Airplay", 0) + 1
        
        if "google_background" in app.remaining_bonjour:
            result["cast"] = result.get("cast", 0) + 1
    return result
            

In [27]:
def without_airplay_and_cast(dataset: List[Result]) -> int:
    result = 0
    for app in dataset:
        if str(ScanType.multicast) in str(app.no_interaction + app.interaction):
            # only airplay or only cast
            # only mDNS
            if len(get_all_multicast_addresses(app.contacted_ip_addresses)) > 1 or "224.0.0.251" not in get_all_multicast_addresses(app.contacted_ip_addresses):
                result = result + 1
            elif len(get_all_multicast_addresses(app.contacted_ip_addresses)) == 1 and "224.0.0.251" in get_all_multicast_addresses(app.contacted_ip_addresses):

                airplay_cast = get_airplay_queries(app.remaining_bonjour) + get_googlecast_queries(app.remaining_bonjour)
                
                if len(app.remaining_bonjour) != len(airplay_cast):
                    result = result + 1
            else:
                result = result + 1
    return result

def only_cast(dataset: List[Result]) -> int:
    result = 0
    for app in dataset:
        if str(ScanType.multicast) in str(app.no_interaction + app.interaction):
            # only airplay or only cast
            # only mDNS
            if len(get_all_multicast_addresses(app.contacted_ip_addresses)) == 1 and "224.0.0.251" in get_all_multicast_addresses(app.contacted_ip_addresses):
                airplay_cast =  get_googlecast_queries(app.remaining_bonjour)
                if len(app.remaining_bonjour) == len(airplay_cast):
                    result = result + 1

    return result

def only_airplay(dataset: List[Result]) -> int:
    result = 0
    for app in dataset:
        if str(ScanType.multicast) in str(app.no_interaction + app.interaction):
            # only airplay or only cast
            # only mDNS
            if len(get_all_multicast_addresses(app.contacted_ip_addresses)) == 1 and "224.0.0.251" in get_all_multicast_addresses(app.contacted_ip_addresses):
                airplay_cast = get_airplay_queries(app.remaining_bonjour)
                if len(app.remaining_bonjour) == len(airplay_cast):
                    result = result + 1
                    print(app.app_id)
                    print(app.log_result)
                    print(app.interaction)
                    print(app.no_interaction)
                    print(app.remaining_bonjour)
                    print(get_all_multicast_addresses(app.contacted_ip_addresses))

    return result



In [28]:
# 
def get_app_for_rerun_other_local_ips(dataset_results: List[Result], rerun_mapping):
    result = []
    for app in dataset_results:
        if len(get_other_local_addresse(app.contacted_ip_addresses)) > 0 and app.app_id not in rerun_mapping:
            result.append(app.app_id)

    return result
            

In [29]:
def load_from_pickle_if_available(file_name: str) -> any:
    """
    Load a variable from a pickle file if it exists.

    Args:
        file_name (str): The name of the pickle file.

    Returns:
        any: The loaded variable from the pickle file, or None if the file doesn't exist.
    """
    with open(file_name, "rb") as file:
        # Deserialize and retrieve the variable from the file
        return pickle.load(file)
    return None

In [32]:

def load_dataset_from_pickle(dataset: str) -> Tuple[List[Analysis], List[str], List[Result]]:
    ds: List[Analysis] = load_from_pickle_if_available(f"{pickle_folder}/{dataset}.pickle")
    ds_result: List[Result] = load_from_pickle_if_available(f"{pickle_folder}/{dataset}_result.pickle")
    return ds, [], ds_result


In [36]:
android_random, _, android_random_result = load_dataset_from_pickle("random")
_, _, android_random_12_result = load_dataset_from_pickle("random_192_168_2_12")
_, _, android_random_147_result = load_dataset_from_pickle("random_192_168_2_147")
android_top, _, android_top_result = load_dataset_from_pickle("ten_mio")
android_matching, _, android_matching_result = load_dataset_from_pickle("matching")
android_all_matching_result = android_matching_result
android_all_random_result = android_random_result + android_random_12_result + android_random_147_result

In [37]:
_, _, android_random_v2_12_result = load_dataset_from_pickle("run_v2_random_192_168_2_12")
_, _, android_matching_v2_12_result = load_dataset_from_pickle("run_v2_matching_192_168_2_12")
android_top_v2, _, android_top_v2_result = load_dataset_from_pickle("run_v2_ten_mio")


In [38]:
remove_wrong_addresses(android_all_random_result, android_random_v2_12_result)
remove_wrong_addresses(android_all_matching_result, android_matching_v2_12_result)
remove_wrong_addresses(android_top_result, android_top_v2_result)

In [39]:
scanning_random = remove_false_positive(android_all_random_result, android_random_v2_12_result)
scanning_matching = remove_false_positive(android_all_matching_result, android_matching_v2_12_result)
scanning_top = remove_false_positive(android_top_result, android_top_v2_result)

scanning_random_all =add_results_from_first_run(android_all_random_result, android_random_v2_12_result)
scanning_matching_all =add_results_from_first_run(android_all_matching_result, android_matching_v2_12_result)
scanning_top_all =add_results_from_first_run(android_top_result, android_top_v2_result)

In [40]:
ios_red, _, ios_red_result = load_dataset_from_pickle("red")
ios_white, _, ios_white_result = load_dataset_from_pickle("white")
ios_white_192_168_2_13, _, ios_white_192_168_2_13_result = load_dataset_from_pickle("white_192_168_2_16")

ios_black, _, ios_black_result = load_dataset_from_pickle("black")
ios_black_192_168_2_10, _, ios_black_192_168_2_10_result = load_dataset_from_pickle("black_192_168_2_10")

ios_black_192_168_2_10_1, _, ios_black_192_168_2_10_1_result = load_dataset_from_pickle("black_192_168_2_10_1")
ios_black_192_168_2_13, _, ios_black_192_168_2_13_result = load_dataset_from_pickle("black_192_168_2_13")
ios_black_192_168_2_16, _, ios_black_192_168_2_16_result = load_dataset_from_pickle("black_192_168_2_16")

white_total = ios_white_192_168_2_13_result + ios_white_result
black_total = ios_black_192_168_2_13_result + ios_black_192_168_2_16_result + ios_black_result + ios_black_192_168_2_10_1_result + ios_black_192_168_2_10_result

In [41]:
ios_top_results = get_matching_results(ios_red_result +white_total + black_total   ,ios_top_ids)
ios_random_results = get_matching_results(ios_red_result +white_total + black_total   ,ios_random_ids)

In [42]:
_, _, red_v2_result = load_dataset_from_pickle("red_v2")
_, _, white_v2_result = load_dataset_from_pickle("white_v2")
_, _, white_v2_16_result = load_dataset_from_pickle("white_v2_192_168_2_16")
_, _, white_v2_10_result = load_dataset_from_pickle("white_v2_192_168_2_10")

_, _, black_v2_result = load_dataset_from_pickle("black_v2")
_, _, black_v2_10_result = load_dataset_from_pickle("black_v2_192_168_2_10")
_, _, black_v2_16_result = load_dataset_from_pickle("black_v2_192_168_2_16")
black_v2_total = black_v2_result + black_v2_10_result + black_v2_16_result
white_v2_result = white_v2_result + white_v2_16_result + white_v2_10_result


In [43]:
remove_wrong_addresses(black_total, black_v2_total)
remove_wrong_addresses(white_total, white_v2_result)
remove_wrong_addresses(ios_red_result, red_v2_result)

In [44]:
# Get all cross platform apps to compute bonjour results
matching_results_1 = util.get_only_matching_results(scanning_matching_all + scanning_random_all + scanning_top_all,  black_total + white_total + ios_red_result, ios_to_android)
matching_results_2 = util.get_only_matching_results(scanning_matching_all + scanning_random_all + scanning_top_all,  black_v2_total + white_v2_result + red_v2_result, ios_to_android)


In [45]:
# only matching results
bonjour_apps,airplay, airprint, google_cast, other  = bonjour_without_string(list(matching_results_1.keys()), list(matching_results_2.keys()))


In [46]:

scanning_black = remove_false_positive(black_total, black_v2_total)
scanning_white = remove_false_positive(white_total, white_v2_result)
scanning_red = remove_false_positive(ios_red_result, red_v2_result)

In [ ]:
# No traffic in dump only cross-platform
manual_no_traffic = []
manual_own_ip = []
for app in matching_results_2.keys():
    if app.app_id in manual_logs_no_traffic:
        if manual_logs_no_traffic[app.app_id] == None:
            manual_no_traffic.append(app.app_id)
        elif str(manual_logs_no_traffic[app.app_id]) == str(ScanType.own_wifi_ip):
            manual_own_ip.append(app.app_id)
print(len(manual_no_traffic))
print(manual_no_traffic)
print(len(manual_own_ip))
print(manual_own_ip)

In [ ]:
add_scan_type_for_logs(black_total, black_v2_total)
add_scan_type_for_logs(white_total, white_v2_result)
add_scan_type_for_logs(ios_red_result, red_v2_result)

add_scan_type_for_logs(black_total, black_v2_total)
add_scan_type_for_logs(white_total, white_v2_result)
add_scan_type_for_logs(ios_red_result, red_v2_result)



In [49]:
ios_scanning_top_results = get_matching_results(black_v2_total+ white_v2_result + red_v2_result   ,ios_top_ids)
ios_scanning_random_results = get_matching_results(black_v2_total+ white_v2_result+ red_v2_result   ,ios_random_ids)

In [50]:
matching_results = util.get_only_matching_results(scanning_matching_all + scanning_random_all + scanning_top_all,  add_results_from_first_run(ios_top_results ,ios_scanning_top_results) + add_results_from_first_run(ios_random_results, ios_scanning_random_results), ios_to_android)

In [ ]:
print(len(airplay))
print(len(airprint))
print(len(google_cast))
print(len(other))

In [ ]:
release_after_permission_other = get_apps_released_after_permission(other, red_mapping | white_mapping | black_mapping)

In [ ]:
release_after_permission_cast = get_apps_released_after_permission(google_cast, red_mapping | white_mapping | black_mapping)

In [ ]:
print(len(release_after_permission_other))
print(release_after_permission_other)
print(len(release_after_permission_cast))
print(release_after_permission_cast)

In [ ]:
set(google_cast + other)- set(release_after_permission_cast) - set(release_after_permission_other)

In [ ]:
print(len(set(google_cast + other)))

In [ ]:
print(len(set(google_cast + other)- set(release_after_permission_cast) - set(release_after_permission_other))) 
print(get_value_format(len(set(google_cast + other)- set(release_after_permission_cast) - set(release_after_permission_other)), len(set(google_cast+other))))

In [ ]:
print(get_value_format(len(release_after_permission_cast), len(set(google_cast + other))))
print(get_value_format(1, len(google_cast+other)))
print(get_value_format(1, len(google_cast+other))) # Hue


In [62]:
# only matching
with_permission, without_permission = (get_prompt_stats(list(matching_results.keys())))


In [ ]:
print(get_value_format(len(with_permission), len(with_permission) + len(without_permission)))
print(get_value_format(len(without_permission), len(with_permission) + len(without_permission)))


In [ ]:
create_table_other_local_network([matching_results.values(), matching_results.keys()], ["Android", "iOS"], format=True)

In [65]:
other_ip_table = create_table_other_local_network([scanning_top_all +scanning_matching_all + scanning_random_all, scanning_top_all, scanning_random_all ,  matching_results.values() , ios_top_results + ios_random_results, ios_top_results, ios_random_results ,matching_results.keys()], ["All", "Top", "Random", "Matching", "All", "Top", "Random" , "Matching"], format=True)
other_ip_table = create_table_other_local_network([scanning_top_all, scanning_random_all ,  matching_results.values() , ios_top_results, ios_random_results ,matching_results.keys()], ["Top", "Random", "Matching", "Top", "Random" , "Matching"], format=True)

In [ ]:
print(other_ip_table.to_latex(index=False))

In [ ]:
android_stats = get_stats(matching_results.values())
ios_stats = get_stats(matching_results.keys())
print(get_value_format(android_stats[0], len(ios_to_android)))
print(get_value_format(ios_stats[0], len(ios_to_android)))

In [68]:
android_stats = get_stats(scanning_top_all +scanning_matching_all + scanning_random_all)
ios_stats = get_stats(ios_scanning_top_results + ios_scanning_random_results)


In [ ]:
overall_table = create_overall_table(scanning_top_all, scanning_random_all, list(matching_results.values()), ios_scanning_top_results, ios_scanning_random_results, list(matching_results.keys()), format=True, only_overall=True, overall_values=[9333, 9961, len(ios_to_android), 9998, 9869 , len(ios_to_android)], airplay_or_multicast=False)
print(overall_table.to_latex())

In [ ]:
overall_table

In [ ]:
# Broadcast and so on don't sum up as apps might access the network on both platforms but do it differentely
one_to_one_table = create_one_to_one_comparison_table(matching_results)

In [ ]:
one_to_one_table

In [ ]:
print(one_to_one_table.to_latex())

In [ ]:
#overall_table_airplay
print(get_background_multicast(matching_results.keys()))
print(get_background_multicast(matching_results.values()))
print(get_value_format(401, len(ios_to_android)))
print(get_value_format(308, len(ios_to_android)))

In [ ]:
create_multicast_table([scanning_top_all, scanning_random_all, list(matching_results.values()),ios_scanning_top_results, ios_scanning_random_results, list(matching_results.keys())], [72, 11,73 , 109,68,117] , ["Top", "Random", "Matching", "Top", "Random", "Matching"], format=True)

In [ ]:
matching_table = create_matching_table(list(matching_results.values()),  list(matching_results.keys()), len(ios_to_android), airplay_or_multicast=False)
print(matching_table.to_latex())

In [ ]:
matching_table

In [ ]:
print(without_airplay_and_cast(matching_results.keys()))
print(without_airplay_and_cast(ios_scanning_top_results))
print(without_airplay_and_cast(ios_scanning_random_results))

In [ ]:
print(without_airplay_and_cast(matching_results.values()))
print(without_airplay_and_cast(scanning_top_all))
print(without_airplay_and_cast(scanning_random_all))

In [ ]:
print(only_cast(matching_results.values()))
print(only_cast(scanning_top_all))
print(only_cast(scanning_random_all))

In [ ]:
print(only_cast(matching_results.keys()))
print(only_cast(ios_scanning_top_results))
print(only_cast(ios_scanning_random_results))

In [ ]:
create_matching_bar_chart(matching_results.values(), matching_results.keys())

In [ ]:
#scans subnet + 1 as well 
other_ip_matching_plot = create_chart_for_other_local_network(matching_results.values(), "192.168.2.252/24", matching_results.keys())
other_ip_matching_plot.tight_layout()


In [ ]:
print(get_apps_with_other_local_ips(matching_results.keys(), "192.168.2.1/24"))
print(get_apps_with_other_local_ips(matching_results.values(), "192.168.2.1/24"))

In [ ]:
matching_android = get_apps_with_other_local_ips(matching_results.values(), "192.168.2.252/24")
print(get_value_format(len(matching_android), len(ios_to_android)))
print(len(matching_results))

In [ ]:
print(len(android_all_matching_result))
print(len(android_top_result))

In [86]:
android_all = get_apps_with_other_local_ips(scanning_top_all +scanning_matching_all + scanning_random_all, "192.168.2.252/24")


In [ ]:
ios_matching = get_apps_with_other_local_ips(matching_results.keys(), "192.168.2.252/24")
print(get_value_format(len(ios_matching), len(ios_to_android)))
print(len(matching_results.keys()))

In [88]:
ios_all = get_number_of_apps_with_other_local_ips(add_results_from_first_run(ios_top_results ,ios_scanning_top_results) + add_results_from_first_run(ios_random_results, ios_scanning_random_results), "192.168.2.252/24")


In [ ]:
print(len(add_results_from_first_run(ios_top_results ,ios_scanning_top_results) + add_results_from_first_run(ios_random_results, ios_scanning_random_results)))


In [ ]:
len(matching_results.values())

In [ ]:
create_bar_chart(matching_results.values(), add_local_and_arp=True)

In [ ]:
create_bar_chart(matching_results.keys(), add_local_and_arp=True)

In [ ]:
android_random_12_result[16].app_id

In [ ]:
create_bar_chart(android_top_result)

In [ ]:
create_bar_chart(android_all_matching_result)

In [ ]:
create_bar_chart(ios_random_results, True)

In [ ]:
create_bar_chart(ios_scanning_random_results)

In [ ]:
create_bar_chart(ios_scanning_top_results)

In [ ]:
create_bar_chart(ios_top_results, True)

In [ ]:
create_bar_chart(black_total, True)

In [ ]:
create_bar_chart(white_total, True)

In [ ]:
create_bar_chart(ios_red_result, iOS=True)

In [ ]:
c = 0
for app in matching_results.keys():
    if str(ScanType.airplay) in str(app.interaction + app.no_interaction):
        c += 1
print(c)

In [ ]:
for app in matching_results.keys():
    if "_googlecast" in  str(app.permission_data.bonjour_services) and str(ScanType.multicast) in str(app.interaction + app.no_interaction) and app.app_id not in google_cast:
        print(app.app_id)
        print(app.interaction)
        print(app.no_interaction)
        print("----")

In [106]:
def create_category_table(matching_results):
    category_ios, category_android, category_both,total = categorice_apps(matching_results)
    header = ["Category", "Overall", "iOS", "Android", "Both"]
    rows = []
    all_keys = set()
    for key in category_ios.keys():
        # do something with key
        all_keys.add(key)

    for key in category_android.keys():
        # do something with key
        all_keys.add(key)

    for key in category_both.keys():
        # do something with key
        all_keys.add(key)

    for key in all_keys:
        row = []
        row.append(key)
        row.append(category_ios.get(key, 0) + category_android.get(key, 0) + category_both.get(key, 0))
        row.append(category_ios.get(key, 0))
        row.append(category_android.get(key, 0))
        row.append(category_both.get(key, 0))
        rows.append(row)

    # Sum up rows where iOS, Android, and Both columns are less than 3
    other_row = ["Other", 0, 0, 0, 0]
    to_remove = []
    for row in rows:
        if row[1] < 3:
            other_row[1] += row[1]
            other_row[2] += row[2]
            other_row[3] += row[3]
            other_row[4] += row[4]
            to_remove.append(row)

    for row in to_remove:
        rows.remove(row)
    rows.append(other_row)

    for row in rows:
        row[1] = get_value_format(row[1], total)
        row[2] = get_value_format(row[2], total)
        row[3] = get_value_format(row[3], total)
        row[4] = get_value_format(row[4], total)

    df = pd.DataFrame(rows, columns=header)
    df.set_index("Category", inplace=True)
    #df = df.sort_values(by="Overall", ascending=False)
    

    return df



In [ ]:
category_table = create_category_table(matching_results)
print(category_table.to_latex())

In [ ]:
print(category_table.to_latex())

In [ ]:
for app in ios_scanning_random_results + ios_scanning_top_results:
    if is_app_scanning(app) and not app.log_result:
        print(app.app_id)
        print(app.no_interaction)